In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install laserembeddings==1.1.2
#!python -m laserembeddings download-models

from tqdm.auto import tqdm
import re
tqdm.pandas()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from laserembeddings import Laser

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import sys
import os
#os.system('python -m laserembeddings download-models')

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=fa8f9cdf56b3ddd45380d9a9827ca88f16c98e27fb671c1d8154291cf8add87c
  Stored in directory: /root/.cache/pip/wheels/d1/ff/0e/e00ff1e22100702ac8b24e709551ae0fb29db9ffc843510a64
Successfully built sacremoses
  Attempting uninstall: sacremoses
    Found existing installation: sacremoses 0.0.53
    Uninstalling sacremoses-0.0.53:
      Successfully uninstalled sacremoses-0.0.53
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.
/kaggle/input/mlt-holy-quran-embeddings/__results__.html

In [2]:
ls /opt/conda/lib/python3.7/site-packages/laserembeddings/data

In [3]:
import site
loc = site.getsitepackages() 
data_path = loc[0]+'/laserembeddings/data'
print(site.getusersitepackages()) # String for user-specific package location)
loc[0]

/root/.local/lib/python3.7/site-packages


'/opt/conda/lib/python3.7/site-packages'

In [4]:

if not os.path.exists(data_path):
    os.makedirs('data')
import shutil
shutil.copy('../input/assets/93langs.fcodes', data_path)
shutil.copy('../input/assets/93langs.fvocab', data_path)
shutil.copy('../input/assets/bilstm.93langs.2018-12-26.pt', data_path)

!ls '/opt/conda/lib/python3.7/site-packages/laserembeddings/data'

93langs.fcodes	93langs.fvocab	bilstm.93langs.2018-12-26.pt


In [5]:
# shutil.copy('/opt/conda/lib/python3.7/site-packages/laserembeddings/data/93langs.fcodes', './')
# shutil.copy('/opt/conda/lib/python3.7/site-packages/laserembeddings/data/93langs.fvocab', './')
# shutil.copy('/opt/conda/lib/python3.7/site-packages/laserembeddings/data/bilstm.93langs.2018-12-26.pt', './')


In [6]:
corpus_emb_quran = np.load('../input/assets/Holy_Quran_mlt_emb.npy')
corpus_emb_hadith = np.load('../input/assets/en_emb_bukhari_muslim.npy')

In [7]:
en_bn_quran_tafsir = pd.read_csv('../input/assets/en_bn_quran_tafsir.csv')

en_bn_bukhari_muslim = pd.read_csv('../input/assets/en_bn_bukhari_muslim.csv')

In [8]:
def similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)
    return np.dot(v1, v2) / n1 / n2

def pairwise_euclidean_dists(x, y):
    dists = -2 * np.matmul(x, y.T)
    dists +=  np.sum(x**2, axis=1)[:, np.newaxis]
    dists += np.sum(y**2, axis=1)
    return  np.sqrt(dists)


laser = Laser()

In [9]:
en_bn_bukhari_muslim.head(1)

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en,narrators,text_bn
0,0,1,Sahih Bukhari,1,1,Revelation,"30418, 20005, 11062, 11213, 11042, 3",حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated Umar bin Al-Khattab: ...,"'Abdullah bin al Zubair bin 'Isa ,Sufyan bin '...",উমর বিন আল-খাত্তাবের বর্ণনায় উমর বিন আল-খাত্ত...


In [10]:
def MLT_Sahih_Hadith_Search_Engine(query,size=10,language = 'en',metric = 'dot',query_embedding=None):

    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = similarity(corpus_emb_hadith, query_embedding)
    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb_hadith, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]
        Top_index_doc = Top_index_doc[:size]

    
    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'source'] = str(en_bn_bukhari_muslim['source'][index])
        find.loc[i,'chapter_no'] = str(en_bn_bukhari_muslim['chapter_no'][index])
        find.loc[i,'hadith_no'] = str(en_bn_bukhari_muslim['hadith_no'][index])
        find.loc[i,'chapter'] = str(en_bn_bukhari_muslim['chapter'][index])
        find.loc[i,'text_ar'] = str(en_bn_bukhari_muslim['text_ar'][index])
        find.loc[i,'text_en'] = str(en_bn_bukhari_muslim['text_en'][index])
        find.loc[i,'text_bn'] = str(en_bn_bukhari_muslim['text_bn'][index])
        find.loc[i,'narrators'] = str(en_bn_bukhari_muslim['narrators'][index])
        

    for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
        find.loc[j,'Score'] = simScore

    return find

In [11]:
en_bn_quran_tafsir.head(1)

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan
0,The Opening,1,1,"In the name of Allah, the Beneficent, the Merc...",In the Name of God the Compassionate the Merciful,بِسۡمِ اللّٰهِ الرَّحۡمٰنِ الرَّحِیۡمِ ﴿۱﴾\nب...,পরম করুণাময় অতি দয়ালু আল্লাহর নামে।,(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর নামে (আরম্...


In [12]:
def Multilingual_Quran_Hadith_Search_Engine(query,size=10,language = 'en',metric = 'dot'):

    query_embedding = laser.embed_sentences(query, lang=language)

    mlt_hadiths = MLT_Sahih_Hadith_Search_Engine(query,size=size,language = 'en',metric = metric,query_embedding=query_embedding)
    
    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = similarity(corpus_emb_quran, query_embedding)
    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb_quran, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]
        Top_index_doc = Top_index_doc[:size]

  
    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'Name'] = str(en_bn_quran_tafsir['Name'][index])
        find.loc[i,'Surah'] = str(en_bn_quran_tafsir['Surah'][index])
        find.loc[i,'Ayat'] = str(en_bn_quran_tafsir['Ayat'][index])
        find.loc[i,'Verse'] = str(en_bn_quran_tafsir['Verse'][index]) 
        find.loc[i,'Tafseer'] = str(en_bn_quran_tafsir['Tafseer'][index]) 
        find.loc[i,'ar_text'] = str(en_bn_quran_tafsir['ar_text'][index])
        
        find.loc[i,'আল_বায়ান'] = str(en_bn_quran_tafsir['আল_বায়ান'][index])
        find.loc[i,'tafsir_bayan'] = str(en_bn_quran_tafsir['tafsir_bayan'][index])

    for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
        find.loc[j,'Score'] = simScore
  
    return find,mlt_hadiths

In [13]:
%%time
find,mlt_hadiths = Multilingual_Quran_Hadith_Search_Engine("who is near to allah?",size=2,language = 'en',metric = 'l2')

CPU times: user 641 ms, sys: 248 ms, total: 889 ms
Wall time: 580 ms


In [14]:
find

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan,Score
0,The Event,56,11,Those are they who will be brought nigh,they are the ones brought near to God,اُولٰٓئِکَ الۡمُقَرَّبُوۡنَ ﴿ۚ۱۱﴾\nاولئک المق...,তারাই সান্নিধ্যপ্রাপ্ত।,(১১) তারাই হবে নৈকট্যপ্রাপ্ত।-তাফসীরে আহসানুল ...,1.642195
1,The Angels,35,17,That is not a hard thing for Allah.,And that is not an arduous thing for God.,وَ مَا ذٰلِکَ عَلَی اللّٰهِ بِعَزِیۡزٍ ﴿۱۷﴾\n...,আর তা আল্লাহর জন্য মোটেই কঠিন নয়।,(১৭) আর আল্লাহর পক্ষে তা কঠিন নয়।-তাফসীরে আহসা...,0.935344


In [15]:
mlt_hadiths

,source,chapter_no,hadith_no,chapter,text_ar,text_en,text_bn,narrators,Score
0,Sahih Bukhari,4,239,Ablutions Wudu',"وبإسناده قال ‏""‏ لا يبولن أحدكم في الماء الدائ...",Narrated `Abdullah: While...,` আবদুল্লাহ বর্ণনা করেছেন: যখন আল্লাহর রসূল (স...,Not Found,4.159265
1,Sahih Bukhari,8,480,Prayers Salat,وقال عاصم بن علي حدثنا عاصم بن محمد، سمعت هذا ...,Narrated `Abdullah: That ...,"`অবদুল্লাহর বর্ণিতঃ যে, আল্লাহর রসূল (সাঃ) বলে...","'Asim bin 'Ali bin 'Asim ,'Asim bin Muhammad b...",4.124897


In [16]:
%%time
find,mlt_hadiths = Multilingual_Quran_Hadith_Search_Engine("আমি কি পুরস্কার পেতে পারি?",language = 'bn',metric = 'l2')
find

CPU times: user 398 ms, sys: 236 ms, total: 634 ms
Wall time: 196 ms


,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan,Score
0,The Pen,68,3,And lo! thine verily will be a reward unfailing.,And assuredly you will have an unfailing reward.,وَ اِنَّ لَکَ لَاَجۡرًا غَیۡرَ مَمۡنُوۡنٍ ۚ﴿۳...,আর নিশ্চয় তোমার জন্য রয়েছে অফুরন্ত পুরস্কার।,(৩) তোমার জন্য অবশ্যই রয়েছে নিরবচ্ছিন্ন পুরস্ক...,1.643042
1,The Star,53,59,"Marvel ye then at this statement,",Do you then marvel in denial at this discourse...,اَفَمِنۡ هٰذَا الۡحَدِیۡثِ تَعۡجَبُوۡنَ ﴿ۙ۵۹﴾...,তোমরা কি এ কথায় বিস্ময় বোধ করছ?,(৫৯) তোমরা কি এই কথায় বিস্ময়বোধ করছ? [1][1] এখ...,0.929772
2,The Calamity,101,10,"Ah, what will convey unto thee what she is! -",And what will show you what it is? that is to ...,وَ مَاۤ اَدۡرٰىکَ مَا هِیَهۡ ﴿ؕ۱۰﴾\nو ما ادرى...,আর তোমাকে কিসে জানাবে হাবিয়া কী?,"১০। কিসে তোমাকে জানাল, তা কি? [1][1] এখানে জাহ...",0.922465
3,The Clot,96,10,A slave when he prayeth ?,a servant namely Muhammad (s) when he prays?,عَبۡدًا اِذَا صَلّٰی ﴿ؕ۱۰﴾\nعبدا اذا صلی ﴿۱۰﴾,"এক বান্দাকে, যখন সে সালাত আদায় করে?",১০। এক বান্দা (রসূলুল্লাহ)কে যখন সে নামায আদায়...,0.918014
4,The Overthrowing,81,26,Whither then go ye ?,So where are you going? So what path are you g...,فَاَیۡنَ تَذۡهَبُوۡنَ ﴿ؕ۲۶﴾\nفاین تذهبون ﴿۲۶﴾,সুতরাং তোমরা কোথায় যাচ্ছ?,"২৬। সুতরাং তোমরা কোথায় চলেছ? [1][1] অর্থাৎ, কে...",0.911182
5,Noah,71,20,That ye may thread the valley-ways thereof.,so that you may follow throughout it spacious ...,لِّتَسۡلُکُوۡا مِنۡهَا سُبُلًا فِجَاجًا ﴿۲۰﴾\...,যেন তোমরা সেখানে প্রশস্ত পথে চলতে পার’।,(২০) যাতে তোমরা চলাফেরা করতে পার প্রশস্ত পথে।’...,0.910477
6,Tâ Hâ,20,28,That they may understand my saying.,so that they may understand what I shall say w...,یَفۡقَهُوۡا قَوۡلِیۡ ﴿۪۲۸﴾\nیفقهوا قولی ﴿۲۸﴾,যাতে তারা আমার কথা বুঝতে পারে’।,(২৮) যাতে তারা আমার কথা বুঝতে পারে।-তাফসীরে আহ...,0.904796
7,The Star,53,4,It is naught save an inspiration that is inspi...,It is but a revelation that is revealed to him,اِنۡ هُوَ اِلَّا وَحۡیٌ یُّوۡحٰی ۙ﴿۴﴾\nان هو ...,"তাতো কেবল ওহী, যা তার প্রতি ওহীরূপে প্রেরণ করা...","(৪) তা তো অহী, যা তার প্রতি প্রত্যাদেশ হয়। [1]...",0.896761
8,The Cloaked One,74,20,"Again (self-)destroyed is he, how he planned! -",Again perish he how he decided!,ثُمَّ قُتِلَ کَیۡفَ قَدَّرَ ﴿ۙ۲۰﴾\nثم قتل کیف...,তারপর সে ধ্বংস হোক! কীভাবে সে সিদ্ধান্ত গ্রহণ ...,(২০) আবার ধ্বংস হোক সে! কেমন করে সে এই সিদ্ধান...,0.892444
9,The Reality,69,20,Surely I knew that I should have to meet my re...,I was truly certain that I would encounter my ...,اِنِّیۡ ظَنَنۡتُ اَنِّیۡ مُلٰقٍ حِسَابِیَهۡ ﴿...,"‘আমার দৃঢ় বিশ্বাস ছিল যে, আমি আমার হিসাবের সম্...","(২০) আমার পূর্ণ বিশ্বাস ছিল যে, আমাকে আমার হিস...",0.891077


In [17]:
mlt_hadiths

,source,chapter_no,hadith_no,chapter,text_ar,text_en,text_bn,narrators,Score
0,Sahih Bukhari,66,5074,Virtues of the Qur'an,حدثنا محمد بن يوسف، حدثنا مالك بن مغول، حدثنا ...,Narrated Talha: I asked `...,তালেহা বর্ণিত: আমি `অবদুল্লাহ বিন আবী `আউফাকে ...,"Muhammad bin Yusuf al Baykandi ,Malik bin Mghw...",4.130790
1,Sahih Bukhari,39,2295,Kafalah,حدثنا أبو عاصم، عن يزيد بن أبي عبيد، عن سلمة ب...,Narrated Salama bin Al-Akwa`: ...,সালমা বিন আল-আকওয়া ` এর কাহিনীঃ একজন মৃত ব্যক...,"al Dahhak bin Mukhlad Abu 'Asim al Nabil ,Yazi...",4.097856
2,Sahih Muslim,13,2378,The Book of Zakat,وحدثنا عبد الله بن عبد الرحمن الدارمي، أخبرنا ...,This hadith has been narrated on the authorit...,এই হাদীসটি জায়েদ কর্তৃক একই কাতার সংরক্ষণকারী...,"'Abdullah bin 'Abdur Rahman al Daarimi ,Yahya ...",4.079525
3,Sahih Bukhari,68,5362,Divorce,حدثنا يحيى بن قزعة، حدثنا مالك، عن ابن شهاب، ع...,Narrated Abu Huraira: A m...,আবু হুরায়রা বর্ণনা করেছেন: একজন লোক নবী (সাঃ)...,"Yahya bin Qz'h ,Imam Maalik ,al Zuhri , ,Sa'id...",3.942238
4,Sahih Bukhari,81,6542,To make the Heart Tender Ar Riqaq,حدثني محمد بن عرعرة، حدثنا شعبة، عن سعد بن إبر...,Narrated `Aisha: The Prop...,`রহঃ আয়েশা (রাঃ) এর কাহিনী `রহঃ নবী (সাঃ) কে ...,"Muhammad bin 'Ar'ara ,Shu'bah bin al Hajjaj ,S...",3.895076
5,Sahih Bukhari,97,7442,Holding Fast to the Qur'an and Sunnah,حدثنا حماد بن حميد، حدثنا عبيد الله بن معاذ، ح...,Narrated Muhammad bin Al-Munkadir: ...,মুহাম্মদ বিন আল-মুন্নাদীর থেকে বর্ণিতঃ আমি দেখ...,"Hmad bin Hameed al Khrasany ,'Ubaidullah bin M...",3.437375
6,Sahih Bukhari,24,1490,Obligatory Charity Tax Zakat,حدثنا عثمان بن أبي شيبة، حدثنا عبدة، عن هشام، ...,Narrated Zainab: (the dau...,বর্ণনা করেছেন জৈনব: (উম সালামার কন্যা) আমার মা...,"'Uthman bin Muhammad Abi Shayba ,'Abdah bin Su...",3.406147
7,Sahih Bukhari,79,6340,Asking Permission,حدثنا موسى بن إسماعيل، حدثنا همام، عن قتادة، ع...,Narrated Mu`adh: While I ...,বর্ণিতঃ যখন আমি রাসূলের সাথে এক রথের সঙ্গী ছিল...,"Musa bin Isma'il al Munqari al Tabudhaki ,Hmam...",3.261773
8,Sahih Bukhari,27,1835,Pilgrims Prevented from Completing the Pilgrim...,حدثنا عبد الله بن يوسف، أخبرنا مالك، عن نافع، ...,Narrated Nafi`: When `Abd...,নাফী ` দ্বারা বর্ণিত: যখন `অবদুল্লাহ বিন `উমার...,"'Abdullah bin Yusuf al Tunaysi ,Imam Maalik ,N...",3.257172
9,Sahih Bukhari,81,6544,To make the Heart Tender Ar Riqaq,حدثنا علي بن عبد الله، حدثنا محمد بن الزبرقان،...,Narrated `Aisha: The Prop...,"`রহঃ আয়েশা থেকে বর্ণিত: নবী (সাঃ) বলেন, ""সৎকর...","'Ali bin al Madini ,Muhammad bin al Zbrqan Abu...",3.145489
